# 03 - Trends + Visualizations

This notebook loads the monthly rollups and generates clean seaborn charts:

- Total month-to-month revenue (all facilities combined)
- Month-to-month revenue split by Encounter Facility

It also marks **July 2023** with a vertical dashed line (when I took over as coordinator).

In [ ]:
import sys
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.viz import plot_total_revenue, plot_revenue_by_facility  # noqa: E402

## Load rollups

In [ ]:
monthly_total = pd.read_csv(ROOT / "outputs" / "monthly_total.csv")
monthly_by_facility = pd.read_csv(ROOT / "outputs" / "monthly_by_facility.csv")

monthly_total["month"] = pd.to_datetime(monthly_total["month"])
monthly_by_facility["month"] = pd.to_datetime(monthly_by_facility["month"])

monthly_total.head()

## Generate charts

In [ ]:
plot_total_revenue(
    monthly_total,
    out_dir=ROOT / "outputs",
    takeover_date="2023-07-01",
)

plot_revenue_by_facility(
    monthly_by_facility,
    out_dir=ROOT / "outputs",
    takeover_date="2023-07-01",
)

print("Saved charts to outputs/")

## Quick sanity checks

In [ ]:
print("Date range (total):", monthly_total["month"].min(), "to", monthly_total["month"].max())
print("Facilities:", sorted(monthly_by_facility["facility"].dropna().unique().tolist()))

## Export a simple pre vs post July 2023 summary

This gives a quick before/after comparison showing how the program changed after I took over.

In [ ]:
takeover = pd.to_datetime("2023-07-01")

pre = monthly_total[monthly_total["month"] < takeover]
post = monthly_total[monthly_total["month"] >= takeover]

summary = pd.DataFrame(
    {
        "period": ["pre_takeover", "post_takeover"],
        "months": [len(pre), len(post)],
        "avg_monthly_revenue": [pre["revenue"].mean(), post["revenue"].mean()],
        "avg_monthly_client_hours": [pre["client_hours"].mean(), post["client_hours"].mean()],
        "avg_monthly_encounters": [pre["encounters"].mean(), post["encounters"].mean()],
    }
)

summary

In [ ]:
# percent change (post vs pre)
pct_change = pd.DataFrame(
    {
        "metric": ["avg_monthly_revenue", "avg_monthly_client_hours", "avg_monthly_encounters"],
        "percent_change": [
            (summary.loc[1, "avg_monthly_revenue"] / summary.loc[0, "avg_monthly_revenue"] - 1) * 100,
            (summary.loc[1, "avg_monthly_client_hours"] / summary.loc[0, "avg_monthly_client_hours"] - 1) * 100,
            (summary.loc[1, "avg_monthly_encounters"] / summary.loc[0, "avg_monthly_encounters"] - 1) * 100,
        ],
    }
)

pct_change

In [ ]:
# Save summary tables
OUT_SUMMARY = ROOT / "outputs" / "takeover_summary.csv"
OUT_PCT = ROOT / "outputs" / "takeover_pct_change.csv"

summary.to_csv(OUT_SUMMARY, index=False)
pct_change.to_csv(OUT_PCT, index=False)

print("Wrote:", OUT_SUMMARY)
print("Wrote:", OUT_PCT)